In [1]:
import os
import io
import sys
import matplotlib.pyplot as plt
import IPython.display as ipd
import pandas as pd
import re
import subprocess
import numpy as np
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib inline

sys.path.append('../src')

In [7]:
bm_dataset_meta_path = '/media/maxim/Databases/BRAVE-MASKS_PROCESSED/features/brave_masks.devel.vad_SileroVoiceActivityDetector.win_4.step_4'
features_type = 'EARLY'
basename = os.path.basename(bm_dataset_meta_path)
meta_folder = basename.replace('brave_masks.', 'brave_masks_{0}_features.'.format(features_type))
meta_folder = bm_dataset_meta_path.replace(os.path.basename(bm_dataset_meta_path), meta_folder)
print(meta_folder)

/media/maxim/Databases/BRAVE-MASKS_PROCESSED/features/brave_masks_EARLY_features.devel.vad_SileroVoiceActivityDetector.win_4.step_4


In [6]:
meta_folder

'/media/maxim/Databases/BRAVE-MASKS_PROCESSED/features/brave_masks_EARLY_featuresdevel.vad_SileroVoiceActivityDetector.win_4.step_4'

In [4]:
os.path.basename('/media/maxim/Databases/BRAVE-MASKS_PROCESSED/features/brave_masks.devel.vad_SileroVoiceActivityDetector.win_4.step_4')

'brave_masks.devel.vad_SileroVoiceActivityDetector.win_4.step_4'

In [5]:
root_dir = '/media/maxim/WesternDigital/ABAWLogs/'

databases_dir = {'C': 'C'}
# databases_dir = {'bravemasks_a': 'BRAVEMASKS/MultimodalAdapted_v2', 'bravemasks_s': 'BRAVEMASKS/MultimodalStandart_v2'}

stats_path = 'logs/stats.csv'

results = {}
for k, db in databases_dir.items():
    results[db] = []
    exp_lists = os.listdir(os.path.join(root_dir, db))
    exp_lists.sort(key=lambda exp_name: os.path.getmtime(os.path.join(root_dir, db, exp_name)), reverse=True)
    for exp_name in exp_lists:
        df = pd.read_csv(os.path.join(root_dir, db, exp_name, stats_path), sep=';')
        best_model = df.iloc[df['abaw_devel_f1'].idxmax()].to_dict()
        for name, v in best_model.items():
            if 'loss' in name or 'epoch' in name or 'experiment_name' in name:
                continue
            else:
                best_model[name] = v * 100

        results[db].append({**{'experiment_name': exp_name}, **best_model})

for k, db in results.items():
    pd.DataFrame(results[k]).to_csv('{0}_results.csv'.format(k.replace(os.sep, '_')), index=False)

In [ ]:
import torch
import torchaudio
import torch.nn.functional as F

from models.audio_transformers import MyAudioModelPyTorch

c_names = ['clear', 'mask']
model_cls = MyAudioModelPyTorch
models = {
    'MyAudioModelPyTorch_WAV2VEC2_BASE': {
        'class': model_cls,
        'args': {
            'inp_size': 16000,
            'transformer_params': ['WAV2VEC2_BASE', 768],
            'out_size': len(c_names),
        }
    }
}


for model_type in models:
    wave, sr = torchaudio.load('/media/maxim/Databases/ComParE_2020_PROCESSED/audio/test/test_00001.wav')    
    wave = wave.unsqueeze(dim=0)

    m1 = models[model_type]['class'](**models[model_type]['args'])
    m2 = models[model_type]['class'](**models[model_type]['args'])
    m2.load_state_dict(torch.load('../models/epoch_111.pth')['model_state_dict'])
    
    m1.eval()
    m2.eval()
    with torch.no_grad():
#         print(m1.stp.register_forward_hook(get_features('stp')))
#         print(m2.stp.register_forward_hook(get_features('stp')))
        print(wave)
        print(wave.shape)
        z1 = m1(wave)
        print(wave)
        print(wave.shape)
        print(m1.extract_fe(wave))
        
        z2 = m2(wave)
        print(m2.extract_fe(wave))
        
        
#         print(z1.)
    
#     m1.fc1 = torch.nn.Identity()
#     m2.fc1 = torch.nn.Identity()

In [29]:
import torch
import torchaudio
import torch.nn.functional as F

from models.audio_transformers import MyAudioModelPyTorch

c_names = ['clear', 'mask']
model_cls = MyAudioModelPyTorch
models = {
    'MyAudioModelPyTorch_WAV2VEC2_BASE': {
        'class': model_cls,
        'args': {
            'inp_size': 16000,
            'transformer_params': ['WAV2VEC2_BASE', 768],
            'out_size': len(c_names),
        }
    }
}

def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook
    
for model_type in models:
    wave, sr = torchaudio.load('/media/maxim/Databases/ComParE_2020_PROCESSED/audio/test/test_00001.wav')    
    wave = wave.unsqueeze(dim=0)
    print(wave.shape)
    m1 = models[model_type]['class'](**models[model_type]['args'])
    m2 = models[model_type]['class'](**models[model_type]['args'])
    m2.load_state_dict(torch.load('../models/epoch_111.pth')['model_state_dict'])
    
    m1.eval()
    m2.eval()
    with torch.no_grad():
#         print(m1.stp.register_forward_hook(get_features('stp')))
#         print(m2.stp.register_forward_hook(get_features('stp')))
        
#         z1 = m1(wave)
        z2 = m2(wave)
        
#         print(z1.)
    
#     m1.fc1 = torch.nn.Identity()
#     m2.fc1 = torch.nn.Identity()
    


torch.Size([1, 1, 16000])
tensor([[ 5.5454e-02,  1.9580e-01, -9.2937e-04,  1.2593e-01,  5.5537e-01,
         -1.1019e-03,  7.6513e-02,  8.6004e-02,  2.9210e-02,  3.4305e-01,
         -2.6372e-04, -1.1736e-03,  3.8961e-02,  5.4420e-01, -2.5212e-03,
         -2.6131e-03, -1.6518e-04, -3.0438e-03, -6.7908e-04,  7.7389e-02,
         -2.2126e-03,  3.4881e-02, -3.1008e-03, -1.4076e-03,  8.3789e-03,
          5.6141e-01,  2.2627e-01,  2.0179e-01,  5.7657e-02, -1.8160e-04,
          1.1726e-01, -1.8823e-03, -2.9266e-03, -8.9208e-04,  2.8181e-01,
          6.0142e-01, -1.2599e-03,  4.1089e-01, -5.6477e-04, -2.0832e-03,
          2.2378e-01,  2.2142e-02, -8.0184e-04, -3.1103e-03, -1.4273e-03,
         -1.1196e-03,  1.2150e-02,  4.4499e-01,  7.3128e-02,  3.3560e-01,
          1.9592e-01, -2.9776e-03,  4.4328e-02, -1.2541e-03, -9.4062e-04,
          3.4709e-01, -1.6911e-03, -4.5928e-04,  4.1844e-01, -3.3037e-03,
          5.7833e-02, -4.9643e-04, -5.9883e-04, -2.8486e-05,  6.8279e-01,
          4.

In [3]:
tf_x = np.random.rand(1, 224, 224, 3).astype('float32')
pth_x = torch.Tensor(np.transpose(tf_x, (0, 3, 1, 2)))

tf_l = tf.keras.models.Model(inputs=[tf_model.get_layer('input_1').input], 
                             outputs=[tf_model.get_layer('dense').output])

tf_out = tf_l(tf_x, training=False).numpy()

pth_model.eval()
with torch.no_grad():
    pth_out = F.softmax(pth_model(pth_x), dim=1).detach().numpy()
np.allclose(tf_out, pth_out, atol=1e-5)

2023-03-02 15:04:40.160942: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500


True

In [6]:
torch.save({
    'epoch': 0,
    'model_state_dict': pth_model.state_dict(),
    'optimizer_state_dict': None,
    'loss': None
}, 'pth_image_mask_detection.h5')

In [7]:
m = LenaResNet()
m.load_state_dict(torch.load('pth_image_mask_detection.h5')['model_state_dict'])
m.eval()
with torch.no_grad():
    m_out = F.softmax(m(pth_x), dim=1).detach().numpy()
    
print(np.allclose(tf_out, m_out, atol=1e-5))
print(np.allclose(m_out, pth_out, atol=1e-5))

True
True


In [10]:
import cv2

def preprocess_input(x, data_format=None, version=1):
    x_temp = np.copy(x)
    if data_format is None:
        data_format = K.image_data_format()
    assert data_format in {'channels_last', 'channels_first'}

    if data_format == 'channels_first':
        x_temp = x_temp[:, ::-1, ...]
        x_temp[:, 0, :, :] -= 91.4953
        x_temp[:, 1, :, :] -= 103.8827
        x_temp[:, 2, :, :] -= 131.0912
    else:
        x_temp = x_temp[..., ::-1]
        x_temp[..., 0] -= 91.4953
        x_temp[..., 1] -= 103.8827
        x_temp[..., 2] -= 131.0912

    return x_temp

t_img = cv2.imread('0_0_0.jpg')
t_img = cv2.cvtColor(t_img, cv2.COLOR_BGR2RGB)
# x = tf.cast(x, np.uint8)
t_img = tf.image.resize(t_img, (224,224))
t_img = tf.keras.preprocessing.image.img_to_array(t_img)
t_img = preprocess_input(t_img)
t_img = np.expand_dims(t_img, axis=0)
dict = {0: 'Without_Mask', 1: 'With_Mask'}
prob = tf_model.predict(t_img)
print(prob)
cl = np.argmax(prob)
print('Predicted class: ', dict[cl])
print('Confidence: {}%'.format(np.round(prob[0][cl]*100, 2)))

1/1 [==============================] - 1s 690ms/step
[[0.03960168 0.9603984 ]]
Predicted class:  With_Mask
Confidence: 96.04%


In [11]:
from PIL import Image
from torchvision import transforms

class PreprocessInput(torch.nn.Module):
    def __init__(self):
        super(PreprocessInput, self).__init__()
        pass
    
    def forward(self, x):
        x = np.asarray(x, dtype='float64')
        x = x[..., ::-1]
        x[..., 0] -= 91.4953
        x[..., 1] -= 103.8827
        x[..., 2] -= 131.0912
        x = torch.Tensor(x.copy())
        return x.permute(2, 0, 1)

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    PreprocessInput(),
])

p_img = Image.open('0_0_0.jpg')
p_img = preprocess(p_img)
p_img = torch.unsqueeze(p_img, 0)

pth_model.eval()
with torch.no_grad():
    res = F.softmax(pth_model(p_img), dim=1).detach().numpy()

print(res)
cl = np.argmax(res)
print('Predicted class: ', dict[cl])
print('Confidence: {}%'.format(np.round(res[0][cl]*100, 2)))

[[0.03922091 0.9607791 ]]
Predicted class:  With_Mask
Confidence: 96.08%
